In [17]:
from sklearn.preprocessing import scale
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.linear_model import Ridge, RidgeClassifier
from sklearn.svm import SVR, SVC
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, RandomizedSearchCV, train_test_split

from scipy.stats import expon

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

In [2]:
# color dict (use: colors['XColor'])

colors = {'DBrwn' : '#8c510a', 'MBrwn' : '#d8b365', 'LBrwn' : '#f6e8c3',
          'DTeal' : '#01665e', 'MTeal' : '#5ab4ac', 'LTeal' : '#c7eae5',
          'DPurp' : '#762a83', 'MPurp' : '#af8dc3', 'LPurp' : '#e7d4e8'
         }

# set params
# save as .mplstyle file and import with: plt.style.use('file')
#mpl.rcParams['lines.linewidth'] = 2
#mpl.rcParams['lines.color'] = 'r'

# colors for seaborn
dark = ["#8c510a", "#01665e", "#762a83"]
med = ["#d8b365", "#5ab4ac", "#af8dc3"]

# Make Predictions wrt Test Set

In [52]:
low_burn_test = '../pkl_trainsets/2jul2018/2jul2018_testset1_nucs_fissact_not-scaled.pkl'
train1 = '../pkl_trainsets/2jul2018/2jul2018_trainset1_nucs_fissact_not-scaled.pkl'
train2 = '../pkl_trainsets/2jul2018/2jul2018_trainset2_nucs_fissact_not-scaled.pkl'
train3 = '../pkl_trainsets/2jul2018/22jul2018_trainset3_nucs_fissact_not-scaled.pkl'
trainXY = pd.read_pickle(train2)
testXY = pd.read_pickle(low_burn_test)

In [53]:
def splitXY(dfXY):
    lbls = ['ReactorType', 'CoolingTime', 'Enrichment', 'Burnup', 'OrigenReactor']
    dfX = dfXY.drop(lbls, axis=1)
    if 'total' in dfX.columns:
        dfX.drop('total', axis=1, inplace=True)
    r_dfY = dfXY.loc[:, lbls[0]]
    c_dfY = dfXY.loc[:, lbls[1]]
    e_dfY = dfXY.loc[:, lbls[2]]
    b_dfY = dfXY.loc[:, lbls[3]]
    #o_dfY = dfXY.loc[:, lbls[4]]
    return dfX, r_dfY, c_dfY, e_dfY, b_dfY

trnX, rY, cY, eY, bY = splitXY(trainXY)
tstX, ry, cy, ey, by = splitXY(testXY)    #lowercase Y denotes testing labels

In [54]:
extra = []
for i in trnX.columns:
    if i not in tstX.columns:
        extra.append(i)
print(extra)
trnX = trnX.drop(extra, axis=1)
print(len(trnX.columns))
print(trnX.columns)
print(len(tstX.columns))
print(tstX.columns)

['cs135', 'pd104', 'pd105', 'pd106', 'pu242']
41
Index(['ba138', 'ce140', 'ce142', 'ce144', 'cs133', 'cs137', 'la139', 'mo100',
       'mo95', 'mo97', 'mo98', 'nd143', 'nd144', 'nd145', 'nd146', 'pr141',
       'pu239', 'pu240', 'pu241', 'rb87', 'rh103', 'ru101', 'ru102', 'ru104',
       'sr88', 'sr90', 'tc99', 'u234', 'u235', 'u236', 'u238', 'xe131',
       'xe132', 'xe134', 'xe136', 'y89', 'zr91', 'zr92', 'zr93', 'zr94',
       'zr96'],
      dtype='object')
41
Index(['ba138', 'ce140', 'ce142', 'ce144', 'cs133', 'cs137', 'la139', 'mo100',
       'mo95', 'mo97', 'mo98', 'nd143', 'nd144', 'nd145', 'nd146', 'pr141',
       'pu239', 'pu240', 'pu241', 'rb87', 'rh103', 'ru101', 'ru102', 'ru104',
       'sr88', 'sr90', 'tc99', 'u234', 'u235', 'u236', 'u238', 'xe131',
       'xe132', 'xe134', 'xe136', 'y89', 'zr91', 'zr92', 'zr93', 'zr94',
       'zr96'],
      dtype='object')


In [55]:
#trainX = scale(trnX)
#testX = scale(tstX)
trainX = trnX
testX = tstX

In [56]:
CV=5
for Y in ('r', 'b', 'c', 'e'):
    trainY = pd.Series()
    # get param names and set ground truth
    if Y == 'c':
        trainY = cY
        testY = cy
        parameter = 'cooling'
    elif Y == 'e': 
        trainY = eY
        testY = ey
        parameter = 'enrichment'
    elif Y == 'b':
        trainY = bY
        testY = by
        parameter = 'burnup'
    else:
        trainY = rY
        testY = ry
        parameter = 'reactor'    
    
    #######################
    # optimize parameters #
    #######################
            
    # initialize learners
    score = 'explained_variance'
    kfold = KFold(n_splits=CV, shuffle=True)
    knn_init = KNeighborsRegressor(n_neighbors=4, weights='distance')
    rr_init = Ridge(alpha=0.001)
    svr_init = SVR(gamma=0.001, C=200)
    if Y is 'r':
        score = 'accuracy'
        kfold = StratifiedKFold(n_splits=CV, shuffle=True)
        knn_init = KNeighborsClassifier(n_neighbors=4, weights='distance')
        rr_init = RidgeClassifier(alpha=0.001, class_weight='balanced')
        svr_init = SVC(gamma=0.001, C=200, class_weight='balanced')
        
    knn_init.fit(trainX, trainY)
    rr_init.fit(trainX, trainY)
    svr_init.fit(trainX, trainY)
    
    ########################
    # run predictions, etc #
    ########################

    scores = ['explained_variance', 'neg_mean_absolute_error']
    if Y is 'r':
        scores = ['accuracy', ]
        
    kpred = knn_init.predict(testX)
    kscr = knn_init.score(testX, testY)
    rpred = rr_init.predict(testX)
    rscr = rr_init.score(testX, testY)
    spred = svr_init.predict(testX)
    sscr = svr_init.score(testX, testY)
    
    print('The {} scores are knn={}, rr={}, and svr={}\n'.format(parameter, kscr, rscr, sscr), flush=True)
    preds_by_alg = pd.DataFrame({'TrueY': testY, 'kNN': kpred, 
                                 'Ridge': rpred, 'SVR': spred}, 
                                  index=testY.index)
    preds_by_alg.to_csv('testset1_v_trainset2_' + parameter + '_predictions.csv')

The reactor scores are knn=0.6179775280898876, rr=0.2808988764044944, and svr=0.8202247191011236

The burnup scores are knn=0.9795543676709813, rr=0.9998361063334887, and svr=0.9172682957144276

The cooling scores are knn=-1.0590040951588833, rr=-223.94143015165483, and svr=0.6184950168338598

The enrichment scores are knn=-0.1386109693036328, rr=-4.278658328192713, and svr=0.5922057054547918

